In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import ssl
import pandas as pd
from utils.model_utils import UNet, store_tested_parameters, train_and_get_validation_loss, train_with_early_stopping
from utils.image_utils import CustomImageDataset
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

torch.hub.set_dir(d = "C:\\Users\\Ugne\\torch\\hub")

data_dir = "images"
device = "cuda"
classes =  ["Person", "Skyscraper", "Car"]

class_mapping = {'Skyscraper': 1, 'Car': 2, 'Person': 3}

def get_dir_for_class(class_name, masks_or_images="images"):
    return data_dir+"/"+class_name.lower()+"/"+masks_or_images

download_dirs = [get_dir_for_class(c) for c in classes]+["images/all"]

In [ ]:
downloaded_annotations = pd.read_csv("downloaded_annotations.csv")

In [240]:
transform = transforms.Compose([
    #transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomAffine(degrees = 20, scale = (0.7,1.3)),
    #transforms.ColorJitter(brightness=.5, hue=.3)
])

target_size = (64,64)

train_dataset = CustomImageDataset(image_dir="images/all", class_annotations=downloaded_annotations, class_mapping=class_mapping, 
                                   transform=transform, dataset_type="train", target_size=target_size)
validation_dataset = CustomImageDataset(image_dir="images/all", class_annotations=downloaded_annotations, class_mapping=class_mapping, 
                                        transform=None, dataset_type="validation", target_size=target_size)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=16)

# Modelling

## Grid search for best batch size and learning rate

We select the optimal batch size and learning rate using Bayesian optimization. The acquisition function uses a Gaussian prior (assumes a normal distribution), which allows the next parameter to be chosen in a smarter way, cutting down on the exploration time.

In [88]:
space = [Integer(1,128, name="batch_size"),
         Real(10**-5, 10**0, "log-uniform", name="learning_rate")]

num_epochs = 5
device = "cuda"

@use_named_args(space)
def objective(**params):
    model = UNet().to(device)
    
    criterion = nn.CrossEntropyLoss()
    
    batch_size = int(params['batch_size'])
    learning_rate = params['learning_rate']
    
    validation_loss = train_and_get_validation_loss(model, train_dataset, validation_dataset, criterion, batch_size, learning_rate)
    
    return validation_loss

res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)

Epoch 1/5, Loss: 0.7179199543866244
Epoch 2/5, Loss: 0.6315036621960727
Epoch 3/5, Loss: 0.6077892411838878
Epoch 4/5, Loss: 0.581497938524593
Epoch 5/5, Loss: 0.5606708813797344
Validation Loss: 0.5728951518734297
Epoch 1/5, Loss: 0.7209381649368688
Epoch 2/5, Loss: 0.6467689699248264
Epoch 3/5, Loss: 0.6063428254503953
Epoch 4/5, Loss: 0.5680287425455294
Epoch 5/5, Loss: 0.5535448813124707
Validation Loss: 0.6104296578301324
Epoch 1/5, Loss: 0.749743123099489
Epoch 2/5, Loss: 0.5289105997895295
Epoch 3/5, Loss: 0.4861830084953668
Epoch 4/5, Loss: 0.46409777706524113
Epoch 5/5, Loss: 0.442688579829234
Validation Loss: 0.564777279065715
Epoch 1/5, Loss: 1.219102081325319
Epoch 2/5, Loss: 0.9603706189879665
Epoch 3/5, Loss: 0.8601658465685668
Epoch 4/5, Loss: 0.7940411771889087
Epoch 5/5, Loss: 0.7422260951112818
Validation Loss: 0.9102703796492683
Epoch 1/5, Loss: 0.6138759932275546
Epoch 2/5, Loss: 0.5020155537936647
Epoch 3/5, Loss: 0.47578804750563736
Epoch 4/5, Loss: 0.462604726522

In [89]:
best_batch_size, best_learning_rate = res_gp.x
best_score = res_gp.fun
all_scores = res_gp.func_vals

store_tested_parameters(res_gp)

best_batch_size, best_learning_rate, best_score

(104, 0.0025112263624837016, 18.446914941072464)

In [ ]:
#best_params_backup = (104, 0.0025112263624837016)

## Train model using optimal batch size and learning rate

In [ ]:
best_model = UNet()
best_model.to(device)
num_epochs = 10
criterion = nn.CrossEntropyLoss()

validation_loss = train_and_get_validation_loss(best_model, train_dataset, validation_dataset, criterion, batch_size=int(best_batch_size), learning_rate=best_learning_rate)

torch.save(best_model.state_dict(), "best_model.pt")

## Training with early stopping

We have selected the optimal batch size and learning rate using a small number of epochs. This only holds under the assumption that the same optimal batch size and learning rate will minimise validation loss for a small number of epochs as it would for a larger number of epochs - this assumption may not always hold in practice.

Now, we should use these parameters and implement early stopping to select the optimal number of epochs.

Early stopping means that if the model does not reach an improvement in the validation set for a set number of epochs (equal to the **patience** parameter), we stop the training early on. This should help us combat overfitting.

In [201]:
best_model_v2 = UNet()
best_model_v2.to(device)

optimal_train_dataloader = DataLoader(train_dataset, batch_size=int(best_batch_size), shuffle=True, drop_last=True)
optimal_optimizer = torch.optim.Adam(best_model_v2.parameters(), lr=best_learning_rate)

train_with_early_stopping(best_model_v2, optimal_optimizer, criterion=criterion, dataloader=optimal_train_dataloader, val_dataloader=validation_dataloader, num_epochs=20, patience=3)

Epoch 1/20, Training Loss: 0.6823548190295696
Epoch 1/20, Validation Loss: 0.6535208410507923
Early stopping counter: 0 out of 3
Epoch 2/20, Training Loss: 0.4969502218067646
Epoch 2/20, Validation Loss: 1.6184678786509745
Early stopping counter: 1 out of 3
Epoch 3/20, Training Loss: 0.46710917428135873
Epoch 3/20, Validation Loss: 0.5965611991044637
Early stopping counter: 0 out of 3
Epoch 4/20, Training Loss: 0.4483248800039291
Epoch 4/20, Validation Loss: 0.579216471394977
Early stopping counter: 0 out of 3
Epoch 5/20, Training Loss: 0.4401305727660656
Epoch 5/20, Validation Loss: 0.5969279593712574
Early stopping counter: 1 out of 3
Epoch 6/20, Training Loss: 0.42813734859228136
Epoch 6/20, Validation Loss: 0.604693432917466
Early stopping counter: 2 out of 3
Epoch 7/20, Training Loss: 0.41338326409459114
Epoch 7/20, Validation Loss: 0.5493906937740944
Early stopping counter: 0 out of 3
Epoch 8/20, Training Loss: 0.40500504747033117
Epoch 8/20, Validation Loss: 0.5417567362656465
E

15.366947948932648